<a href="https://colab.research.google.com/github/iamatul1214/NLP/blob/main/BERT_Using_Ktrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import ktrain
except:
  print("installing ktrain")
  !pip install ktrain
  import ktrain

from google.colab import files

installing ktrain
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 22.3 MB 76.9 MB/s 
     |████████████████████████████████| 981 kB 26.8 MB/s 
     |████████████████████████████████| 263 kB 65.0 MB/s 
     |████████████████████████████████| 2.8 MB 50.1 MB/s 
     |████████████████████████████████| 1.2 MB 56.6 MB/s 
     |████████████████████████████████| 468 kB 63.4 MB/s 
     |████████████████████████████████| 596 kB 66.4 MB/s 
     |████████████████████████████████| 3.3 MB 63.3 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 880 kB 57.8 MB/s 
  Created wheel for ktrain: filename=ktrain-0.31.2-py3-none-any.whl size=25313063 sha256=54a1d0cd68cd9d190dbaef75cb4b2d992fcb678bf097b32f9bdafaab9bacdd79
  Stored in directory: /root/.cache/pip/wheels/0a/b3/dd/512a01ea0b4f31d96163518b58361bb72d3b

In [2]:
from ktrain import text
from sklearn.datasets import fetch_20newsgroups

In [3]:
newsgroups_train = fetch_20newsgroups(subset='train')

categories = newsgroups_train.target_names
categories

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
train_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

In [5]:
X_train, y_train, X_test, y_test = train_data.data, train_data.target, test_data.data, test_data.target

In [9]:
IDX = 0

print(f"training dataset instance example = \n {X_train[IDX]}")
print(f"category = \n {categories[y_train[IDX]]}")

training dataset instance example = 
 From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





category = 
 rec.autos


In [10]:
# preprocess  

## Here we will download pretrained BERT model 

train, val, preprocess = text.texts_from_array(
    x_train=X_train,
    y_train=y_train,
    x_test=X_test,
    y_test=y_test,
    class_names=train_data.target_names,
    preprocess_mode='bert',
    maxlen=350
    )

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [11]:
model = text.text_classifier('bert', train_data=train, preproc=preprocess)


Is Multi-Label? False
maxlen is 350
done.


In [12]:
learner = ktrain.get_learner(model, train_data=train, val_data=val, batch_size=4)

In [13]:
## training the model
LEARNING_RATE = 3e-5
EPOCHS = 4
learner.fit_onecycle(lr=LEARNING_RATE, epochs=EPOCHS)



begin training using onecycle policy with max lr of 3e-05...
Epoch 1/4
2829/2829 [==============================] - 1412s 490ms/step - loss: 1.1381 - accuracy: 0.6693 - val_loss: 0.6400 - val_accuracy: 0.8097
Epoch 2/4
2829/2829 [==============================] - 1359s 480ms/step - loss: 0.3822 - accuracy: 0.8839 - val_loss: 0.5970 - val_accuracy: 0.8383
Epoch 3/4
2829/2829 [==============================] - 1356s 480ms/step - loss: 0.1848 - accuracy: 0.9454 - val_loss: 0.5527 - val_accuracy: 0.8630
Epoch 4/4
2829/2829 [==============================] - 1357s 480ms/step - loss: 0.0494 - accuracy: 0.9856 - val_loss: 0.5283 - val_accuracy: 0.8704


In [14]:
predictor = ktrain.get_predictor(model, preproc=preprocess)

In [15]:
text_in = "There is a problem with my computer. The screen of the monitor becomes blurry"
predictor.predict(texts=text_in)

'comp.sys.ibm.pc.hardware'

In [16]:
text_in = "There was another case of a gun violence in a school of the USA"
predictor.predict(texts=text_in)

'talk.politics.guns'